In [9]:
import pyodbc

def create_sql_connection():
    try:
        conn = pyodbc.connect(
            'DRIVER={SQL Server};'
            'SERVER=192.168.101.219;'
            'DATABASE=DataforAnalysis;'
            'UID=DATALYZER;'
            'PWD=NMB54321'
        )
        
        return conn
    except pyodbc.Error as e:
        print("Error connecting to SQL Server:", e)
        return None
    
def create_sql_Component_Master():
    try:
        conn = pyodbc.connect(
            'DRIVER={SQL Server};'
            'SERVER=192.168.101.219;'
            'DATABASE=Component_Master;'
            'UID=DATALYZER;'
            'PWD=NMB54321'
        )
        
        return conn
    except pyodbc.Error as e:
        print("Error connecting to SQL Server:", e)
        return None


In [10]:
from flask import Flask, jsonify
import json
import seaborn as sns
import numpy as np
import base64
import datetime
from flask import send_file, Flask, jsonify, request
from flask_cors import CORS
import pyodbc
import math
import matplotlib.pyplot as plt
import pandas as pd
import pymssql
import matplotlib
matplotlib.use('Agg')


app = Flask(__name__)
cors = CORS(app)


import pandas as pd

def fetch_data(model, Line, start, end, selecteKPOV, selecteKPIV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    print(f"001fetch_data",selecteKPIV)
    query_template = """
        SELECT {selecteKPOV}, {selecteKPIV}
        FROM [Diecast].[dbo].[Pivot]
        JOIN [TransportData].[dbo].[Matching_Auto_Unit1] ON [Pivot].Diecast_S_N = [Matching_Auto_Unit1].Barcode_Base
        JOIN [DataforAnalysis].[dbo].[DataML_Test] ON [DataML_Test].Barcode_motor = [Matching_Auto_Unit1].Barcode_Motor
        WHERE [DataML_Test].[Model] = ? 
        AND [DataML_Test].[Line] = ? 
        AND [DataML_Test].[Date] BETWEEN ? AND ?
        {kpi_conditions}
    """
    # Split the comma-separated selecteKPIV into a list
    selecteKPIV_list = selecteKPIV.split(',')

    # Generate conditions for selecteKPIV columns
    kpi_conditions = ""
    for column in selecteKPIV_list:
        kpi_conditions += f"AND {column.strip()} IS NOT NULL\n"

    # Execute the query with parameters and fetch data
    query = query_template.format(selecteKPOV=selecteKPOV, selecteKPIV=','.join(selecteKPIV_list), kpi_conditions=kpi_conditions)
    datasets = pd.read_sql(query, conn, params=(model, Line, start, end))

    return datasets

In [11]:
def BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call the summary_describe function
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV)
    print(type(minKPOV) )
    print(type(maxKPOV) )
    # Modify the selected column as needed
    data[selecteKPOV] = pd.cut(
        data[selecteKPOV],
        bins=[-np.inf, float(minKPOV), float(maxKPOV), np.inf],
        labels=['fail_low', 'Pass', 'fail_high']
    )

    # Replace 'fail_low' and 'fail_high' with 'fail'
    data[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the original data
    count_before_smote = dict(Counter(data[selecteKPOV]))

    k = 3
    X = data.loc[:, data.columns != selecteKPOV]  # Use selecteKPOV here
    y = data[selecteKPOV]  # Use selecteKPOV here

    # Sampling strategy
    sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
    X_res, y_res = sm.fit_resample(X, y)

    # Concatenate the resampled data
    datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

    # Replace 'fail_low' and 'fail_high' with 'fail' in the resampled data
    datasets[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the resampled data
    count_after_smote = dict(Counter(datasets[selecteKPOV]))

    # Convert the DataFrame to a JSON object
    json_data = datasets.to_json(orient='records')

    # Create a dictionary to include the count results in the JSON response
    response_data = {
        "data": json_data,
        "count_before_smote": count_before_smote,
        "count_after_smote": count_after_smote
      
    }
    return response_data

import pandas as pd
import pyodbc
import json
import pandas as pd



In [12]:
def data_bin(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    
    # สร้างรายการเพื่อเก็บ DataFrames สำหรับแต่ละ selecteKPIV
    result_datasets = []
    
    for KPIV in selecteKPIV:
        query_template = f"""
        with set1 as (SELECT [id]
          ,[Fullname]
          ,[Model]
          ,[Parameter]
          ,[USL]
          ,[LSL]
          ,CL
          ,[USL]-[LSL] as "X"
          ,([USL]-[LSL])/6 as "Y"
          ,[Part]
          ,[Machine]
          ,[empNumber]
          ,[createdAt]
          ,[updatedAt]
      FROM [Component_Master].[dbo].[Master_matchings]
      )
      select 
        [Model]
        ,[Parameter]
        ,[LSL] - 0.0001 as "LCL_3"
        ,[LSL] + 1 * y - 0.0001 as "LCL_2"
        ,[LSL] + 2 * y - 0.0001 as "LCL_1"
        ,[LSL] + 3 * y - 0.0001 as "CL"
        ,[LSL] + 4 * y - 0.0001 as "UCL_1"
        ,[LSL] + 5 * y - 0.0001 as "UCL_2"
        ,[LSL] + 6 * y as "UCL_3"
      from set1
      where Model='{model}' and parameter = '{KPIV}'
        """
        query = query_template
        datasets_bin = pd.read_sql(query, conn)
        result_datasets.append(datasets_bin)
    
    # รวม DataFrames ในรายการเป็น DataFrame เดียวกัน
    combined_dataframe = pd.concat(result_datasets, axis=0, ignore_index=True)

    # แปลง DataFrame เป็น JSON
    combined_json = combined_dataframe.to_json(orient='records')
    
    # ส่ง JSON ผ่าน API response
    return jsonify(combined_json)


In [13]:
model = 'LONGSP'
line = '3-6'
start = '2023-08-01'
end = '2023-09-14'
selecteKPOV = ['Projection1']
selecteKPIV = ['Parallelism_Stack', 'Set_Dimension_Stack', 'P1_Attractive_1']
minKPOV = 0.4648
maxKPOV = 0.5664

# เรียกใช้งานฟังก์ชัน data_bin
combined_dataframe = data_bin(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV)
print(combined_dataframe)

C:\Users\IT\AppData\Local\Temp\ipykernel_17420\4087391506.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets_bin = pd.read_sql(query, conn)
C:\Users\IT\AppData\Local\Temp\ipykernel_17420\4087391506.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets_bin = pd.read_sql(query, conn)
C:\Users\IT\AppData\Local\Temp\ipykernel_17420\4087391506.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets_bin = pd.read_sql(query, conn)


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.